In [2]:
!pip install ipywidgets
!ls
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
CITATION.cff     classify         models           runs
LICENSE          copy.ipynb       requirements.txt setup.cfg
__pycache__      detect.py        resimg.jpg       tutorial.ipynb
benchmarks.py    export.py        result.png       utils
best.pt          hubconf.py       result.txt
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import cv2
import siot
'''
siot.init(client_id="",server="192.168.43.235",port=1883,user="siot",password="dfrobot")
siot.connect()
siot.loop()
siot.getsubscribe(topic="siot/xyz")
'''
objlist=[]
class jxbc:
    def __init__(self,name,acc,x1,x2,y1,y2):
        self.name=name
        self.acc=acc
        self.cenx=(int(x1)+int(x2))/2
        self.ceny=(int(y1)+int(y2))/2
class obj:
    def __init__(self,name,acc,x1,x2,y1,y2):
        self.name=name
        self.acc=acc
        self.disx=0-int(((int(y1)+int(y2))/2-200)/15)
        self.disy=int((((1280-int(x1))+(1280-int(x2)))/2-840)/15)

def modeluse(img):
    global objlist
    c=cv2.imread(img)
    cv2.imwrite("result.png",c)
    !python detect.py --source result.png --weight best.pt
    with open('result.txt','r')as f:
        a=f.readlines()
        a=[a[i].split() for i in range(len(a))]
    for i in range(len(a)):
        if a[i][0]=='jxb':
            jxb=jxbc(a[i][0],a[i][1],a[i][2],a[i][4],a[i][3],a[i][5])
        else:
            objlist.append(obj(a[i][0],a[i][1],a[i][2],a[i][4],a[i][3],a[i][5]))
    strs=""
    for i in range(len(objlist)):
        strs=strs+"选择{}：{}\n".format(i+1,objlist[i].name)
    return strs,"resimg.jpg"

def send0():
    n=0
    strn="{},{},0".format(objlist[n].disx/100,objlist[n].disy/100)
    siot.publish_save(topic="siot/xyz", data=strn)
    return strn
def send1():
    n=1
    strn="{},{},0".format(objlist[n].disx/100,objlist[n].disy/100)
    siot.publish_save(topic="siot/xyz", data=strn)
    return strn
def send2():
    n=2
    strn="{},{},0".format(objlist[n].disx/100,objlist[n].disy/100)
    siot.publish_save(topic="siot/xyz", data=strn)
    return strn
def send3():
    n=3
    strn="{},{},0.1".format(objlist[n].disx/100,objlist[n].disy/100)
    siot.publish_save(topic="siot/xyz", data=strn)
    return strn

with gr.Blocks() as demo:
    image = gr.Image(type="filepath",source="webcam")
    submit_btn = gr.Button("submit")
    resimg= gr.Image(type="filepath")
    object_text=gr.Textbox(label="请选择")
    submit_btn.click(fn=modeluse, inputs=image, outputs=[object_text,resimg])
    choose1_btn=gr.Button("选择1")
    choose2_btn=gr.Button("选择2")
    choose3_btn=gr.Button("选择3")
    choose4_btn=gr.Button("选择4")
    chooseout=gr.Textbox(label="执行结果")
    choose1_btn.click(fn=send0,outputs=chooseout)
    choose2_btn.click(fn=send1,outputs=chooseout)
    choose3_btn.click(fn=send2,outputs=chooseout)
    choose4_btn.click(fn=send3,outputs=chooseout)

demo.launch(share=True)